# Text BBoxes from PDFs

## Function Definitions

### Extractor Functions

In [ ]:
from typing import Any, Optional, Union

import fitz  # PyMuPDF
import numpy as np
import pandas as pd

In [ ]:
# This is not relevant if you're not using OCR. Included just in case.

# This sets a global flag that makes text bboxes more similar to OCR bboxes.
# The default behavior is to set bbox height to the line height, which is considerably
# taller (~1.3x) than the character height. OCR will fit to character height.
# See the note at the bottom of this section:
# https://pymupdf.readthedocs.io/en/latest/textpage.html#span-dictionary
fitz.TOOLS.set_small_glyph_heights(True)

In [ ]:
# copied from well gas project wellgas/features/extract_text.py
def extract_pdf_data_from_page(page: fitz.Page) -> dict[str, pd.DataFrame]:
    """Parse PDF page data."""
    contents = _parse_page_contents(page)
    meta = {
        "rotation_degrees": [page.rotation],
        "origin_x_pdf_coord": [page.rect[0]],
        "origin_y_pdf_coord": [page.rect[1]],
        "width_pdf_coord": [page.rect[2] - page.rect[0]],
        "height_pdf_coord": [page.rect[3] - page.rect[1]],
        "has_images": [not contents["image"].empty],
        "has_text": [not contents["pdf_text"].empty],
        "page_num": [page.number],
    }
    if not contents["image"].empty:
        img_area = (
            contents["image"]
            .eval(
                "((bottom_right_x_pdf - top_left_x_pdf)"
                " * (bottom_right_y_pdf - top_left_y_pdf))"
            )
            .sum()
        )
    else:
        img_area = 0
    total_area = meta["width_pdf_coord"][0] * meta["height_pdf_coord"][0]

    meta["image_area_frac"] = [np.float32(img_area / total_area)]
    meta_df = pd.DataFrame(meta).astype(
        {
            "rotation_degrees": np.int16,
            "origin_x_pdf_coord": np.float32,
            "origin_y_pdf_coord": np.float32,
            "width_pdf_coord": np.float32,
            "height_pdf_coord": np.float32,
            "has_images": "boolean",
            "has_text": "boolean",
            "page_num": np.int16,
            "image_area_frac": np.float32,
        }
    )
    meta = dict(page=meta_df)
    for df in contents.values():  # add ID fields
        if not df.empty:
            df["page_num"] = np.int16(page.number)
    return contents | meta


def _parse_page_contents(page: fitz.Page) -> dict[str, pd.DataFrame]:
    """Parse page contents using fitz.TextPage."""
    flags = fitz.TEXTFLAGS_DICT
    textpage = page.get_textpage(flags=flags)
    content = textpage.extractDICT()
    images = []
    text = []
    for block in content["blocks"]:
        if block["type"] == 0:
            parsed = _parse_text_block(block)
            if not parsed.empty:  # can happen when all blocks are whitespace
                text.append(parsed)
        elif block["type"] == 1:
            images.append(_parse_image_block(block))
        else:
            raise ValueError(f"Unknown block type: {block['type']}")
    if text:
        text = pd.concat(text, axis=0, ignore_index=True)
    else:
        text = pd.DataFrame()
    if images:
        images = pd.concat(
            (pd.DataFrame(image) for image in images), axis=0, ignore_index=True
        )
    else:
        images = pd.DataFrame()
    return dict(pdf_text=text, image=images)


def _parse_image_block(img_block: dict[str, Any]) -> pd.DataFrame:
    """Parse an image block from a fitz.TextPage.extractDICT() output."""
    top_left_x_pdf, top_left_y_pdf, bottom_right_x_pdf, bottom_right_y_pdf = img_block[
        "bbox"
    ]
    dpi = min(
        img_block["xres"], img_block["yres"]
    )  # should be equal; min() just in case
    out = pd.DataFrame(
        {
            "img_num": [img_block["number"]],
            "dpi": [dpi],
            "top_left_x_pdf": [top_left_x_pdf],
            "top_left_y_pdf": [top_left_y_pdf],
            "bottom_right_x_pdf": [bottom_right_x_pdf],
            "bottom_right_y_pdf": [bottom_right_y_pdf],
        }
    ).astype(
        {
            "img_num": np.int16,
            "dpi": np.int16,
            "top_left_x_pdf": np.float32,
            "top_left_y_pdf": np.float32,
            "bottom_right_x_pdf": np.float32,
            "bottom_right_y_pdf": np.float32,
        }
    )
    return out


def _parse_text_block(text_block: dict[str, Any]) -> pd.DataFrame:
    """Parse a text block from a fitz.TextPage.extractDICT() output."""
    # block and line bboxes have no information additional to the text bboxes
    out = dict(
        block_num=[],
        line_num=[],
        font_size=[],
        font=[],
        color_srgb=[],
        text=[],
        top_left_x_pdf=[],
        top_left_y_pdf=[],
        bottom_right_x_pdf=[],
        bottom_right_y_pdf=[],
    )
    block_num = text_block["number"]
    for line_num, line in enumerate(text_block["lines"]):
        for span in line["spans"]:  # no numbering, differentiated by font properties
            if span["text"].isspace():  # ignore whitespace spans
                continue
            if _frac_normal_ascii(span["text"]) < 0.7 and len(span["text"]) > 3:
                # Long non-ASCII is likely bad encoding. It has created problems like
                # unbalanced quotes or unquoted \r and \n in CSVs
                out["text"].append("")
            else:
                out["text"].append(span["text"])
            out["block_num"].append(block_num)
            out["line_num"].append(line_num)
            out["font_size"].append(span["size"])
            out["font"].append(span["font"])
            out["color_srgb"].append(span["color"])
            out["top_left_x_pdf"].append(span["bbox"][0])
            out["top_left_y_pdf"].append(span["bbox"][1])
            out["bottom_right_x_pdf"].append(span["bbox"][2])
            out["bottom_right_y_pdf"].append(span["bbox"][3])
    out = pd.DataFrame(out).astype(
        {
            "block_num": np.int16,
            "line_num": np.int16,
            "font_size": np.float32,
            "font": "string",
            "color_srgb": np.int32,
            "text": "string",
            "top_left_x_pdf": np.float32,
            "top_left_y_pdf": np.float32,
            "bottom_right_x_pdf": np.float32,
            "bottom_right_y_pdf": np.float32,
        }
    )
    return out


def _frac_normal_ascii(text: Union[str, bytes]) -> float:
    """Fraction of characters that are normal ASCII characters."""
    # normal characters, from space to tilde, plus whitespace
    # see https://www.asciitable.com/
    sum_ = 0
    if isinstance(text, bytes):
        text = text.decode("utf-8")
    for char in text:
        if (32 <= ord(char) <= 126) or char in "\t\n":
            sum_ += 1
    return sum_ / len(text)


### Image Rendering Functions

In [ ]:
import fitz  # PyMuPDF
from PIL import Image

In [ ]:

def _render_page(
    pg: fitz.Page, dpi=150, clip: Optional[fitz.Rect] = None
) -> Image.Image:
    """Render a page of a PDF as a PIL.Image object.

    Args:
        pg (fitz.Page): a page of a PDF
        dpi (int, optional): image resolution in pixels per inch. Defaults to 150.
        clip (Optional[fitz.Rect], optional): Optionally render only a subset of the
            page. Defined in PDF coordinates. Defaults to None, which renders the
            full page.

    Returns:
        Image.Image: PDF page rendered as a PIL.Image object
    """
    # 300 dpi is what tesseract recommends. PaddleOCR seems to do fine with half that.
    render: fitz.Pixmap = pg.get_pixmap(dpi=dpi, clip=clip)  # type: ignore
    img = _pil_img_from_pixmap(render)
    return img


def _pil_img_from_pixmap(pix: fitz.Pixmap) -> Image.Image:
    """Convert pyMuPDF Pixmap object to PIL.Image object.

    For some reason pyMuPDF (aka fitz) lets you save images using PIL, but does not
    have any function to convert to PIL objects. Clearly they do this conversion
    internally; they should just expose it. Instead, I had to copy it out from their
    source code.

    Args:
        pix (fitz.Pixmap): a rendered Pixmap

    Returns:
        Image: a PIL.Image object
    """
    # pyMuPDF source code on GitHub is all in SWIG (some kind of C to python code
    # generator) and is unreadable to me. So you have to inspect your local .py files.
    # Adapted from the Pixmap.pil_save method in python3.9/site-packages/fitz/fitz.py
    # I just replaced instances of "self" with "pix"
    cspace = pix.colorspace
    if cspace is None:
        mode = "L"
    elif cspace.n == 1:
        mode = "L" if pix.alpha == 0 else "LA"
    elif cspace.n == 3:
        mode = "RGB" if pix.alpha == 0 else "RGBA"
    else:
        mode = "CMYK"

    img = Image.frombytes(mode, (pix.width, pix.height), pix.samples)
    return img


## Demo

### Load a PDF

In [ ]:
from pathlib import Path
src_path = Path("./wisconsin_electric.pdf")
assert src_path.exists()

In [ ]:
# from file
doc = fitz.Document(str(src_path))
doc.is_pdf

In [ ]:
# from bytes
_bytes = src_path.read_bytes()
from io import BytesIO
doc = fitz.open(stream=BytesIO(_bytes), filetype="pdf")
doc.is_pdf

### Extract Text Bboxes

In [ ]:
pg = doc[0]
extracted = extract_pdf_data_from_page(pg)
extracted.keys()

In [ ]:
txt = extracted['pdf_text']
img_info = extracted['image']
pg_meta = extracted['page']
txt.shape, img_info.shape, pg_meta.shape

In [ ]:
txt.head()

In [ ]:
txt

In [ ]:
img_info  # no images in this doc. Do any Ex 21 docs have images? I guess we'll find out.

In [ ]:
pg_meta

### Render Image

#### Full page

In [ ]:
pg_img = _render_page(pg, dpi=50)  # small dpi for notebook display
pg_img

#### Render a subset of the page

In [ ]:
# here I create a clipping rectangle as a convex hull around some text.
# But you could define the clipping rectangle however you want. It's just a box.
coord_cols = ['top_left_x_pdf', 'top_left_y_pdf', 'bottom_right_x_pdf', 'bottom_right_y_pdf']
subset_text = txt.loc[6:10,coord_cols]  # an artibrary subset of text
convex_hull = subset_text.agg(dict(zip(coord_cols, ['min', 'min', 'max', 'max'])))
convex_hull

In [ ]:
clip = fitz.Rect(*convex_hull) # x0, y0, x1, y1
clip

In [ ]:
_render_page(pg, dpi=50, clip=clip)

## Visualize the Bboxes

Images are awesome but take a little extra housekeeping. There are many ways to represent an image and every library uses a different one as its standard. There is constant conversion: OpenCV expects BGR in numpy arrays, matplotlib expects RGB in numpy arrays, and PIL defaults to RGB but uses its own array structure (not numpy).

Additionally, we have to convert between pixel coordinates and PDF coordinates.

### Viz Functions

In [ ]:

import cv2
import matplotlib.pyplot as plt

PDF_POINTS_PER_INCH = 72  # I believe this is standard for all PDFs

def pil_to_cv2(image: Image.Image) -> np.ndarray:  # noqa: C901
    """Convert a PIL Image to an OpenCV image (numpy array)."""
    # copied from https://gist.github.com/panzi/1ceac1cb30bb6b3450aa5227c02eedd3
    # This covers the common modes, is not exhaustive.
    mode = image.mode
    new_image: np.ndarray
    if mode == "1":
        new_image = np.array(image, dtype=np.uint8)
        new_image *= 255
    elif mode == "L":
        new_image = np.array(image, dtype=np.uint8)
    elif mode == "LA" or mode == "La":
        new_image = np.array(image.convert("RGBA"), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    elif mode == "RGB":
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif mode == "RGBA":
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    elif mode == "LAB":
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_LAB2BGR)
    elif mode == "HSV":
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_HSV2BGR)
    elif mode == "YCbCr":
        # XXX: not sure if YCbCr == YCrCb
        new_image = np.array(image, dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_YCrCb2BGR)
    elif mode == "P" or mode == "CMYK":
        new_image = np.array(image.convert("RGB"), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif mode == "PA" or mode == "Pa":
        new_image = np.array(image.convert("RGBA"), dtype=np.uint8)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    else:
        raise ValueError(f"unhandled image color mode: {mode}")

    return new_image


def cv2_to_pil(img: np.ndarray) -> Image.Image:
    """Create PIL Image from numpy pixel array."""
    if len(img.shape) == 2:  # single channel, AKA grayscale
        return Image.fromarray(img)
    else:  # only handle BGR for now
        return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


def display_img_array(img: np.ndarray, figsize=(5, 5), **kwargs):
    """Plot image array for jupyter sessions."""
    plt.figure(figsize=figsize)
    if len(img.shape) == 2:  # grayscale
        return plt.imshow(img, cmap="gray", vmin=0, vmax=255, **kwargs)
    else:
        return plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), **kwargs)


def overlay_bboxes(
    img: np.ndarray, bboxes: np.ndarray, color=(255, 0, 0)
) -> np.ndarray:
    """Overlay bounding boxes of shape N x 4 (x0, y0, x1, y1) on an image."""
    img = img.copy()
    for box in np.round(bboxes, 0).astype(np.int32):  # float to int just in case:
        x0, y0, x1, y1 = box
        cv2.rectangle(img, (x0, y0), (x1, y1), color=color, thickness=1)
    return img


def pdf_coords_to_pixel_coords(coords: np.ndarray, dpi: int) -> np.ndarray:
    """Convert PDF coordinates to pixel coordinates."""
    # For arbitrary PDFs you would need to subtract the origin in PDF coordinates,
    # but since you create these PDFs, you know the origin is (0, 0).
    out = coords * dpi / PDF_POINTS_PER_INCH
    return out

In [ ]:
dpi = 70
bboxes = pdf_coords_to_pixel_coords(txt[coord_cols].values, dpi=dpi)
img = np.array(_render_page(pg, dpi=dpi))
display_img = overlay_bboxes(img, bboxes)
display_img_array(display_img, figsize=(10, 10))

Some of the bboxes in the table look a little low? Not sure if that is a bbox scaling issue or a rendering issue or a PDF generation issue. I'll leave that as an exercise for the reader ;)